# Análise de Dados em Criptomoedas: Detecção de fraudes

---



**Professora:** Profa. Dra. Isadora Garcia Ferrão

📧 isadoraferrao@usp.br

🌐 [linkedin](https://www.linkedin.com/in/isadora-ferrao/)

🔗 [Instagram](https://www.instagram.com/isadora.ferrao/)

**Descrição dos Dados:** Os dados foram coletados de fontes públicas de monitoramento de carteiras e transações na rede Ethereum, como Etherscan, Dune Analytics e outras plataformas abertas. São dados reais e anonimizados, amplamente utilizados em estudos de detecção de fraude e análise de comportamento no mercado de criptomoedas. Cada linha representa uma carteira e suas métricas de atividade.

**Objetivo:** Construir um modelo preditivo capaz de identificar, com base nas métricas de transações, quando uma carteira apresenta comportamento anômalo, caracterizando uma potencial fraude.



## **Análise exploratória**

In [ ]:
# Importando bibliotecas necessárias


In [ ]:
# Configuração básica dos gráficos


In [ ]:
# --- 1. Carregamento e limpeza dos dados ---


In [ ]:
# Remover espaços em branco dos nomes das colunas


In [ ]:
# Cria a coluna 'is_fraud' de forma direta:
# 1 se 'Sent tnx' for maior que (média + 2 * desvio padrão), senão 0
df["is_fraud"] = (df["Sent tnx"] > (df["Sent tnx"].mean() + 2 * df["Sent tnx"].std())).astype(int)

In [ ]:
# Exibir informações iniciais para diagnóstico


## **Visualizando Correlações e Distribuições**

In [ ]:
# Defina as colunas relevantes para a correlação


In [ ]:
# Histogramas para visualizar a distribuição de 'Sent tnx'
plt.figure(figsize=(10, 6))
sns.histplot(df["Sent tnx"], bins=30, kde=True)
plt.xlabel("Sent tnx")
plt.title("Distribuição de 'Sent tnx'")
plt.show()



In [ ]:
# Boxplot para identificar outliers em 'ERC20 avg val sent'
plt.figure(figsize=(8, 6))
sns.boxplot(x=df["ERC20 avg val sent"])
plt.xlabel("ERC20 avg val sent")
plt.title("Boxplot de 'ERC20 avg val sent'")
plt.show()


## **BASELINE**

In [ ]:
# Calcular média e desvio padrão para 'Sent tnx'


In [ ]:
# Marcar como anomalia as transações com 'Sent tnx' acima do threshold


In [ ]:
# Exibir histograma com o threshold



In [ ]:
# Método Baseline:

## **AULA 2 :)**